In [1]:
import onnx, tf2onnx

import tensorflow as tf
from tensorflow.keras.activations import linear
from tensorflow.keras.models import load_model
from tensorflow.python.compiler.tensorrt import trt_convert as trt

2023-09-29 17:41:03.567407: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 17:42:11.682107: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/modules/software/cuDNN/8.4.1.50-CUDA-11.7.0/lib:/apps/modules/software/Python/3.10.4-GCCcore-11.3.0/lib:/apps/modules/software/libffi/3.4.2-GCCcore-11.3.0/lib64:/apps/modules/software/GMP/6.2.1-GCCcore-11.3.0/lib:/apps/modules/software/SQLite/3.38.3-GCCcore-11.3.0/lib:/apps/modules/software/Tcl/8.6.12-GCCcore-11.3.0/lib:/apps/modules/software/HDF5/1.13.1-gompi-2022a/lib:/apps/modules/

In [2]:
# Load model0
model_path = "/fred/oz016/damon/gauss_ml_1_detector/models/model_test1resnetconv1d_8to1noise_2class_snr5to50_same.h5"
# model_path = "/fred/oz016/damon/gauss_ml_1_detector/models/model_testtest_1det_resnet_same.h5"
model = load_model(model_path)
model.layers[-1].activation = linear
model.compile()
print(model.inputs[0].dtype)

model.save('tf_test')

<dtype: 'float32'>


INFO:tensorflow:Assets written to: tf_test/assets


INFO:tensorflow:Assets written to: tf_test/assets


In [3]:
!python -m tf2onnx.convert --saved-model tf_test --output temp.onnx

/apps/modules/software/Python/3.10.4-GCCcore-11.3.0/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-09-29 17:44:46,758 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2023-09-29 17:44:50,524 - INFO - Signatures found in model: [serving_default].
2023-09-29 17:44:50,524 - WARNING - '--signature_def' not specified, using first signature: serving_default
2023-09-29 17:44:50,525 - INFO - Output names: ['dense_4']
2023-09-29 17:44:51,083 - INFO - Using tensorflow=2.11.0, onnx=1.14.0, tf2onnx=1.14.0/8f8d49
2023-09-29 17:44:51,084 - INFO - Using opset <onnx, 15>
2023-09-29 17:44:51,137 - INFO - Computed 0 values for constant folding
2023-09-29 17:44:51,139 - INFO - Computed 0 values for constant folding
2023-09-29 17:44:51,148 - INFO - Computed 0 values for constant folding
2023-09-2

In [4]:
BATCH_SIZE = 512

onnx_model = onnx.load_model('temp.onnx')

inputs = onnx_model.graph.input
for input in inputs:
    dim1 = input.type.tensor_type.shape.dim[0]
    dim1.dim_value = BATCH_SIZE
    
model_name = "model-512.onnx"
onnx.save_model(onnx_model, model_name)